In [144]:
import sys

sys.path.append('../')

In [145]:
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

from config.regressors import NNRegressor

In [146]:
class AutoEncoder(nn.Module):
    def __init__(self, latent_dim):
        super().__init__()
        
        # Encoder: [3 → 64 → 32 → 16 → latent_dim]
        self.encoder = nn.Sequential(
            nn.Linear(3, 16),
            nn.SELU(),
            nn.Linear(16, 8),
            nn.SELU(),
            nn.Linear(8, 4),
            nn.SELU(),
            nn.Linear(4, latent_dim)  # No activation
        )
        
        # Decoder: [latent_dim → 16 → 32 → 64 → 3]
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, 4),
            nn.SELU(),
            nn.Linear(4, 8),
            nn.SELU(),
            nn.Linear(8, 16),
            nn.SELU(),
            nn.Linear(16, 3)  # No activation
        )
        
        self._init_weights()
    
    def _init_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                # LeCun normal initialization for SELU
                nn.init.normal_(m.weight, 0, std=1. / np.sqrt(m.in_features))
                if m.bias is not None:
                    nn.init.zeros_(m.bias)
    
    def forward(self, x):
        return self.decoder(self.encoder(x))

In [147]:
def trainer(bone, latent_dim):
    # ----------------------------
    # 1. Configuration
    # ----------------------------
    device = 'mps' if torch.backends.mps.is_available() else 'cpu'
    print(f"Using device: {device}")

    # Hyperparameters
    batch_size     = 4096
    learning_rate  = 1e-3
    weight_decay   = 1e-4
    max_epochs     = 100
    patience       = 20
    num_workers    = 4  # >0 for parallel loading

    # ----------------------------
    # 2. Prepare Data (CPU‑only)
    # ----------------------------
    # Assume Y is your NumPy array of shape [N, 3*B]
    PATH = f'/Users/marco/PROJECTS/data/'
    Y = np.load(PATH + 'Y_all.npy')
    Y_bone = Y[:, bone*3:(bone+1)*3]

    Yb_cpu   = torch.as_tensor(Y_bone, dtype=torch.float32)  # stays on CPU

    dataset      = TensorDataset(Yb_cpu, Yb_cpu)
    train_loader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=True,
        num_workers=num_workers,
        persistent_workers=True
    )
    val_loader   = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=num_workers,
        persistent_workers=True
    )

    # instantiate with your chosen latent dimension
    model = AutoEncoder(latent_dim).to(device)

    # ----------------------------
    # 4. Loss & Optimizer
    # ----------------------------
    criterion = nn.MSELoss()
    optimizer = optim.Adam(
        model.parameters(),
        lr=learning_rate,
        weight_decay=weight_decay
    )

    # ----------------------------
    # 5. Training Loop w/ Early Stopping
    # ----------------------------
    best_val_loss     = float('inf')
    epochs_no_improve = 0

    for epoch in range(1, max_epochs+1):
        # —— Training —— 
        model.train()
        running_loss = 0.0

        for xb_cpu, yb_cpu in train_loader:
            # move batch to MPS (or CUDA) on-the-fly
            xb = xb_cpu.to(device, non_blocking=True)
            yb = yb_cpu.to(device, non_blocking=True)

            optimizer.zero_grad()
            preds = model(xb)              
            loss  = criterion(preds, yb)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        avg_train_loss = running_loss / len(train_loader)

        # —— Validation ——
        model.eval()
        running_val = 0.0
        with torch.no_grad():
            for xb_cpu, yb_cpu in val_loader:
                xb = xb_cpu.to(device, non_blocking=True)
                yb = yb_cpu.to(device, non_blocking=True)
                running_val += criterion(model(xb), yb).item()
        avg_val_loss = running_val / len(val_loader)

        print(f"Epoch {epoch:02d}: Train MSE = {avg_train_loss:.6f} | Val MSE = {avg_val_loss:.6f}")

        # —— Early Stopping ——
        if avg_val_loss < best_val_loss:
            best_val_loss     = avg_val_loss
            epochs_no_improve = 0
            torch.save(model.state_dict(), f"best_models/best_model_bone{bone}.pth")
            print("  → New best model saved")
        else:
            epochs_no_improve += 1
            if epochs_no_improve >= patience:
                print("Early stopping triggered")
                break

In [148]:
def encode(Y):
    latent_dims = {
        0: 1, 1: 1, 2: 1,
        3: 1, 4: 2, 5: 1,
        6: 1, 7: 2, 8: 1,
        9: 1, 10: 2, 11: 1,
        12: 1, 13: 1, 14: 2,
        15: 1, 16: 1,}
    
    Y_shape = Y.shape
    
    Y_enc = []
    for bone in range(17):
        latent_dim = latent_dims[bone]
        model = AutoEncoder(latent_dim=latent_dim).to('mps')
        model.load_state_dict(torch.load(f'best_models/best_model_bone{bone}.pth'))
        
        to_encode = Y[..., 3*bone:3*(bone+1)].reshape(-1,3)
        to_encode_tensor = torch.tensor(
            to_encode,
            dtype=torch.float,
            device='mps')
        encoded_tensor = model.encoder(to_encode_tensor)
        Y_enc.append(encoded_tensor.cpu().detach().numpy())

    Y_enc = np.hstack(Y_enc)
    Y_enc = Y_enc.reshape(*Y_shape[:-1], 21)
    
    return Y_enc

In [149]:
def decode(Y):
    latent_dims = {
        0: 1, 1: 1, 2: 1,
        3: 1, 4: 2, 5: 1,
        6: 1, 7: 2, 8: 1,
        9: 1, 10: 2, 11: 1,
        12: 1, 13: 1, 14: 2,
        15: 1, 16: 1,}
    
    Y_shape = Y.shape
    
    current = 0
    Y_dec = []
    for bone in range(17):
        latent_dim = latent_dims[bone]
        model = AutoEncoder(latent_dim=latent_dim).to('mps')

        model.load_state_dict(torch.load(f'best_models/best_model_bone{bone}.pth'))

        to_decode = Y[..., current:current + latent_dim].reshape(-1, latent_dim)
        current += latent_dim

        to_decode_tensor = torch.tensor(
            to_decode,
            dtype=torch.float,
            device='mps')
        decoded_tensor = model.decoder(to_decode_tensor)
        Y_dec.append(decoded_tensor.cpu().detach().numpy())
        
    Y_dec = np.hstack(Y_dec)
    Y_dec = Y_dec.reshape(*Y_shape[:-1], 51)

    return Y_dec

### Test

In [195]:
from config.transformers import TimeDomainTransformer

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

from sklearn.kernel_ridge import KernelRidge
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor

import pyriemann
import pyriemann.regression

from xgboost import XGBRegressor

baseline_kr = Pipeline(
    [
        ('feature_extraction', TimeDomainTransformer(sigma_mpr=0.3)),
        ('scaler', StandardScaler()),
        ('regressor', KernelRidge(
            alpha = 0.01,
            gamma = 0.01,
            kernel='laplacian'))
    ]
)

baseline_knn = Pipeline(
    [
        ('feature_extraction', TimeDomainTransformer(sigma_mpr=0.3)),
        ('scaler', StandardScaler()),
        ('regressor', KNeighborsRegressor(
            n_neighbors = 5))
    ]
)

baseline_rf = Pipeline(
    [
        ('feature_extraction', TimeDomainTransformer(sigma_mpr=0.3)),
        ('scaler', StandardScaler()),
        ('regressor', RandomForestRegressor(
            n_estimators = 50,
            max_depth = 10,
            random_state=42))
    ]
)

baseline_xgb = Pipeline(
    [
        ('feature_extraction', TimeDomainTransformer(sigma_mpr=0.3)),
        ('scaler', StandardScaler()),
        ('regressor', XGBRegressor())
    ]
)

riem_kr = Pipeline(
    [
        ('feature_extraction', pyriemann.estimation.Covariances()),
        ('transformation', pyriemann.tangentspace.TangentSpace(
            metric = 'riemann',
            tsupdate = True)),
        ('scaler', StandardScaler()),
        ('regressor', KernelRidge(
            alpha = 0.01,
            gamma = 0.01,
            kernel='laplacian'))
    ]
)

riem_knn = Pipeline(
    [
        ('feature_extraction', pyriemann.estimation.Covariances()),
        ('transformation', pyriemann.tangentspace.TangentSpace(
            metric = 'riemann',
            tsupdate = True)),
        ('scaler', StandardScaler()),
        ('regressor', KNeighborsRegressor(
            n_neighbors = 5))
    ]
)

riem_rf = Pipeline(
    [
        ('feature_extraction', pyriemann.estimation.Covariances()),
        ('transformation', pyriemann.tangentspace.TangentSpace(
            metric = 'riemann',
            tsupdate = True)),
        ('scaler', StandardScaler()),
        ('regressor', RandomForestRegressor(
            n_estimators = 50,
            max_depth = 10,
            random_state=42))
    ]
)

riem_xgb = Pipeline(
    [
        ('feature_extraction', pyriemann.estimation.Covariances()),
        ('transformation', pyriemann.tangentspace.TangentSpace(
            metric = 'riemann',
            tsupdate = True)),
        ('scaler', StandardScaler()),
        ('regressor', XGBRegressor())
    ]
)

In [151]:
import numpy as np
from config.transformers import TimeWindowTransformer, LabelWindowExtractor

PATH = f'/Users/marco/PROJECTS/data/'
# PATH = r'C:\Users\gianm\Documents\Uni\Big Data\F422\project\data\\'

DATASET = 'guided'
X_freemoves = np.load(PATH + f'{DATASET}/{DATASET}_dataset_X.npy')         # shape (5, 8, 230000)
Y_freemoves = np.load(PATH + f'{DATASET}/{DATASET}_dataset_Y.npy')         # shape (5, 51, 230000)

# define parameters
size = 500
step = 250

# initialize transformers
tw_transformer = TimeWindowTransformer(size=size, step=step)
label_extractor = LabelWindowExtractor(size=size, step=step)

# apply transformations
X_freemoves_windows = tw_transformer.transform(X_freemoves)           # shape: (5, n_windows, 8, 500)
Y_freemoves_labels = label_extractor.transform(Y_freemoves)

In [152]:
from config.validation import cross_validate_pipeline, RMSE, NMSE

X_folds = np.vstack(X_freemoves_windows[:4])
X_test = X_freemoves_windows[4]

Y_folds = np.vstack(Y_freemoves_labels[:4])
Y_test = Y_freemoves_labels[4]

In [153]:
from config.regressors import StackingRegressor, VotingRegressor
from config.transformers import TimeWindowTransformer, LabelWindowExtractor

In [154]:
from sklearn.neighbors import RadiusNeighborsRegressor

In [209]:
pipeline = StackingRegressor(
    estimators = [
        baseline_kr,
        baseline_knn,
        baseline_rf,
        # baseline_xgb,
        # riem_kr,
        riem_knn,
        # riem_rf,
        # riem_xgb
    ],
    end_estimator = KNeighborsRegressor(
        n_neighbors=7,
        p=1)
    # end_estimator = RadiusNeighborsRegressor(radius=12)
)

# pipeline = VotingRegressor(
#     estimators = [
#         baseline_kr,
#         baseline_knn,
#         baseline_rf,
#         riem_knn,
#         riem_rf
#     ]
#     # end_estimator = RandomForestRegressor(
#     #         n_estimators = 50,
#     #         max_depth = 10)
# )

# feature scaler
targ_scal = StandardScaler()

# actual regression
Y_folds = np.vstack(Y_freemoves_labels[:4])

Y_folds = targ_scal.fit_transform(Y_folds) # scaling targets

pipeline.fit(X_folds, Y_folds)
Y_pred = pipeline.predict(X_test)

Y_pred = targ_scal.inverse_transform(Y_pred) # scaling targets back

print('RMSE without autoencoders:', RMSE(Y_pred, Y_test))

# encoded regression
Y_folds = np.vstack(Y_freemoves_labels[:4])

Y_folds_enc = encode(Y_folds) # shape (..., 21) rather than (..., 51)
Y_folds_enc = targ_scal.fit_transform(Y_folds_enc) # scaling targets

pipeline.fit(X_folds, Y_folds_enc)
Y_enc_pred = pipeline.predict(X_test) # shape (..., 21) rather than (..., 51)

Y_enc_pred = targ_scal.inverse_transform(Y_enc_pred) # scaling targets back
Y_dec_pred = decode(Y_enc_pred) # back to shape (..., 51)


print('RMSE with autoencoders:', RMSE(Y_dec_pred, Y_test))
print('')

RMSE without autoencoders: 3.354287340881273
RMSE with autoencoders: 3.2728642920137223



## Prediction generation

In [156]:
import pandas as pd

In [210]:
model = StackingRegressor(
    estimators = [
        baseline_kr,
        baseline_knn,
        baseline_rf,
        # riem_kr,
        riem_knn,
        # riem_rf
    ],
    end_estimator = KNeighborsRegressor(
        n_neighbors=7,
        p=1
    )
)

confirmation = input('Have you inserted the correct model name and the expected RMSE?')

model_name = 'prova_autoencoders'
expected_rmse = '1000'

step_prediction = 50 # step for prediction

# preparing the training data
tw_extractor = TimeWindowTransformer(size = 500, step = step_prediction)
label_extractor = LabelWindowExtractor(size = 500, step = step_prediction)

DATASET = 'guided'

X = np.load(PATH + f'{DATASET}/{DATASET}_dataset_X.npy')
Y = np.load(PATH + f'{DATASET}/{DATASET}_dataset_Y.npy')

DATASET = 'freemoves'
XX = np.load(PATH + f'{DATASET}/{DATASET}_dataset_X.npy')
YY = np.load(PATH + f'{DATASET}/{DATASET}_dataset_Y.npy')

XX = XX[..., -230000:]
YY = YY[..., -230000:]

X = np.vstack([X, XX])
Y = np.vstack([Y, YY])

X_windows = tw_extractor.transform(X)
Y_labels = label_extractor.transform(Y)

# encoding
print('Encoding...')
Y_labels = encode(Y_labels)
print('Encoded.')

# stacking the sessions 
X_train = X_windows.reshape(-1, *X_windows.shape[2:])
Y_train = Y_labels.reshape(-1, *Y_labels.shape[2:])

# scaling the target
targ_scal = StandardScaler()
Y_train = targ_scal.fit_transform(Y_train)

# training
print('Training...')
model.fit(X_train, Y_train)
print('Trained.')

# predicting
DATASET = 'guided'
X_test = np.load(PATH + f'{DATASET}/{DATASET}_testset_X.npy')
X_test = X_test.reshape(-1, *X_windows.shape[2:])
print('Predicting...')
Y_pred = model.predict(X_test)
print('Predicted.')

# scaling back
Y_pred = targ_scal.inverse_transform(Y_pred)

# decoding
print('Decoding...')
Y_pred = decode(Y_pred)
print('Decoded.')

# saving
file_name = 'guided_encoding_scaling_meta_50.npy'

np.save(file_name, Y_pred)

Y_pred_guided = np.load('guided_encoding_scaling_meta_50.npy')
Y_pred_freemoves = np.load('dann_steps_50_rmse_10.npy')

Y_pred = np.vstack([Y_pred_guided, Y_pred_freemoves])
Y_pred_df = pd.DataFrame(Y_pred)
Y_pred_df.to_csv('very_last_try.csv', index=False, header=False)

Encoding...
Encoded.
Training...
Trained.
Predicting...
Predicted.
Decoding...
Decoded.
